# change_data(input_file,out_file,r) 정의 :

input_file의 데이터를 읽고 각 로고 데이터의 r 비율을 바꿔 out_file로 저장

- input_file : .mat 파일 이름 (경로 포함)

- output_file : . mat 파일 이름 (경로 포함)

- r : [0,1] 의 소수

In [1]:
# .mat 파일 읽어서 각 로고 데이터의 r 비율을 바꾼다.

def change_data(input_file,out_file,r):
    
    # import 모듈
    from scipy import io
    import numpy as np
    import copy
    
    # .mat 파일 읽기
    mat_file = io.loadmat(input_file)
    
    # .mat 파일 내용 읽기
    groundtruth=mat_file['groundtruth']
    
    file_mat=groundtruth['file'][0]
    bboxes_mat=groundtruth['bboxes'][0]
    name_mat=groundtruth['name'][0]
    
    # 로고 종류 담을 리스트
    logo_lst=[]
    
    # 바꾼 데이터 개수
    num=0
    
    # 로고 종류 뽑기
    for logo in name_mat:

        if logo not in logo_lst:

            logo_lst.append(logo)

    # 각 로고마다 r 비율로 데이터 변형
    for logo in logo_lst:
        
        # 로고 수
        num_logo=sum(name_mat==logo)

        # 로고의 원래 인덱스
        # tuple 의미 있는 건 orig_ind[0]
        orig_ind=np.where(name_mat==logo)

        # 변형할 개수
        num_choice=(int)(num_logo*r)
        
        num+=num_choice

        # 변형할 데이터의 인덱스
        ind=np.random.choice(orig_ind[0],num_choice,replace=False)

        # 위,왼쪽,아래쪽,오른쪽 변형할 개수 저장
        sides=np.array([0,0,0,0])

        # num_choice에서 4로 나누는데 나머지 있을 수 있으므로,
        # 고르게 하기 위해 다음과 같이 처리
        base= int(num_choice/4)
        
        sides=sides+base
        
        residue=int(num_choice%4)
        
        # 나머지를 어디에 더할지 랜덤으로 정함
        res=[0,1,2,3]

        choice=np.random.choice(res,residue,replace=False)

        sides[choice]+=1
        
        # ind 앞에서부터 차례대로 진행

        #'Up' side
        for i in range(sides[0]):

            # 변형할 인덱스 저장하고 ind에서 지운다 
            indx=ind[0]
            ind=np.delete(ind,0,0)
     
            # 바운딩 박스 여러 개일 수 있으므로 각각 적용 
            for bbox in bboxes_mat[indx]:

                # h
                bbox[3]/=2

                # y
                bbox[1]+=(bbox[3]/2)     

        # 'Left' side
        for i in range(sides[1]):

            indx=ind[0]
            ind=np.delete(ind,0,0)

            for bbox in bboxes_mat[indx]:

                # w
                bbox[2]/=2

                # x
                bbox[0]-=(bbox[2]/2)

        #'Down' side
        for i in range(sides[2]):


            indx=ind[0]
            ind=np.delete(ind,0,0)

            for bbox in bboxes_mat[indx]:

                # h
                bbox[3]/=2

                # y
                bbox[1]-=(bbox[3]/2)

        #'Right' side
        for i in range(sides[3]):

            indx=ind[0]
            ind=np.delete(ind,0,0)
            
            for bbox in bboxes_mat[indx]:

                # w
                bbox[2]=bbox[2]/2

                # x
                bbox[0]+=(bbox[2]/2)

    # 결과물 저장
    out=io.savemat(out_file,{'groundtruth':groundtruth})
    
    return num
    

# 실행 하기 :

In [2]:
num=change_data('groundtruth.mat','h.mat',0.2)

# 확인하기 : 

In [3]:
from scipy import io
import numpy as np

import copy

In [4]:
mat_file = io.loadmat('groundtruth.mat')

hi_file=io.loadmat('h.mat')

In [5]:
num2=0

for i in range(len(mat_file['groundtruth']['bboxes'][0])):
    
    if np.any(mat_file['groundtruth']['bboxes'][0][i]-hi_file['groundtruth']['bboxes'][0][i]):
        #print(i)
        num2+=1
        
print(num)
print(num2)

1552
1552
